In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("BatterySimV3.py"))
parent = os.path.dirname(os.path.dirname(current))
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DPModel2
from Logic import logic_bat, logic_DP, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


## Running the logic models and DP

In [3]:
#"Input" variables
avg_yearly_cost_dict = {}
N = 24*7
bat_capacity=13
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

In [4]:
# Define battery with capcacity 0, same as no battery
battery_no_bat = Battery(max_capacity=0,max_charge=0)

series_no_battery = logic_rollout(merged.loc[Start:End].copy(), battery_no_bat, logic_bat)

charge_list_no_bat = battery_no_bat.charge_list

avg_yearly_cost_dict["No battery"] = (print_price_summary(series_no_battery), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat>0]), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat<=0]))

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  802.0  DKK
Total emissions for period:  64.0  kg
Average cost per year is: 42077.0 DKK
Average emissions per year is: 3354.0 kg
Number of kwh purchased in the period: 456.3
Average number of kwh purchased per year: 23951.651497005987
Average number of kwh sold per year: 34334.3748502994


In [5]:
logic_series_print(series_no_battery)

hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8971,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957,  0.5890,  3.0853
    5:   0.8254,  0.1850, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022,  0.6290,  3.7143
    6:   0.8373,  0.2410, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326,  0.8435,  4.5578
    7:

  141:   2.4617,  0.1860, -5.9000, -5.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.5239,672.1365,  1.0974, 55.0821
  142:   2.3804,  0.1240, -6.6000, -6.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 15.7109,687.8474,  0.8184, 55.9005
  143:   1.9674,  0.1130, -6.0000, -6.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.8042,699.6516,  0.6780, 56.5785
  144:   1.9364,  0.1300, -6.0000, -6.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.6182,711.2698,  0.7800, 57.3585
  145:   1.7616,  0.1460, -4.9000, -4.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.6319,719.9017,  0.7154, 58.0739
  146:   1.5989,  0.1550, -4.9000, -4.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.8346,727.7363,  0.7595, 58.8334
  147:   1.6151,  0.1330, -4.4000, -4.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.1062,734.8425,  0.5852, 59.4186
  148:   1.5158,  0.1370, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.3053,740.1478,  0.4795, 59.8981
  149:  

In [6]:
# Define battery with capcacity
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].copy(), battery, logic_bat)

charge_list_bat = battery.charge_list

avg_yearly_cost_dict["Simple logic"] = (print_price_summary(series_battery), 
                                        np.sum(charge_list_bat[charge_list_bat>0]), 
                                        np.sum(charge_list_bat[charge_list_bat<=0]))

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  652.0  DKK
Total emissions for period:  57.0  kg
Average cost per year is: 34211.0 DKK
Average emissions per year is: 2976.0 kg
Number of kwh purchased in the period: 382.40000000000003
Average number of kwh purchased per year: 20072.56526946108
Average number of kwh sold per year: 28297.907784431136


In [7]:
logic_series_print(series_battery)

hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8971,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957,  0.5890,  3.0853
    5:   0.8254,  0.1850, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022,  0.6290,  3.7143
    6:   0.8373,  0.2410, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326,  0.8435,  4.5578
    7:

  162:   2.0204,  0.1170,  0.4000,  0.0000,  0.0000,  0.4000, 13.0000, 12.3000, 12.7000, -0.0000,616.5925,  0.0819, 55.0867
  163:   2.3369,  0.1040, -1.8000,  0.0000,  0.0000, -1.8000, 12.7000, 12.0000, 10.2000, -0.0000,616.5925,  0.0728, 55.1595
  164:   2.4341,  0.1030, -5.0000,  0.0000,  0.0000, -5.0000, 10.2000,  9.6000,  4.6000, -0.0000,616.5925,  0.0618, 55.2213
  165:   2.4007,  0.0990, -6.2000, -1.9000,  0.0000, -4.3000,  4.6000,  4.3000,  0.0000,  4.5613,621.1538,  0.2178, 55.4391
  166:   2.4107,  0.0860, -6.5000, -6.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 15.6694,636.8232,  0.5590, 55.9981
  167:   2.1943,  0.1030, -6.8000, -6.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.9214,651.7446,  0.7004, 56.6985


In [8]:
DPbat = Battery(max_capacity=bat_capacity)

DP = DPModel(N, Start, End, house, merged, deepcopy(DPbat))

J1, pi = DP_stochastic(DP)

0   

In [9]:
J, traj, actions = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=0.0)
J

603.42326095286

In [10]:
series_DP_battery = logic_rollout(merged.loc[Start:End].copy(), DPbat, logic_DP, actions)

charge_list_DP_bat = DPbat.charge_list

avg_yearly_cost_dict["DP"] = (print_price_summary(series_DP_battery), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat>0]), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat<=0]))

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  603.0  DKK
Total emissions for period:  58.0  kg
Average cost per year is: 31674.0 DKK
Average emissions per year is: 3039.0 kg
Number of kwh purchased in the period: 393.5000000000001
Average number of kwh purchased per year: 20655.21556886228
Average number of kwh sold per year: 29206.00239520958


In [11]:
logic_series_print(series_DP_battery)

hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8971,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957,  0.5890,  3.0853
    5:   0.8254,  0.1850, -3.4000, -3.4000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022,  0.6290,  3.7143
    6:   0.8373,  0.2410, -3.5000, -3.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326,  0.8435,  4.5578
    7:

  160:   1.6417,  0.0700, 17.2000, 16.5000,  0.0000,  0.7000, 13.0000, 12.3000, 13.0000, -2.7088,571.2581,  0.0000, 56.1940
  161:   2.0018,  0.0950, 14.7000, 14.0000,  0.0000,  0.7000, 13.0000, 12.3000, 13.0000, -2.8026,568.4555,  0.0000, 56.1940
  162:   2.0204,  0.1170,  0.4000, -0.3000,  0.3000,  0.7000, 13.0000, 12.3000, 13.0000,  0.6061,569.0616,  0.1170, 56.3110
  163:   2.3369,  0.1040, -1.8000, -1.1000,  0.0000, -0.7000, 13.0000, 12.3000, 11.6000,  2.5705,571.6322,  0.1872, 56.4982
  164:   2.4341,  0.1030, -5.0000,  0.0000,  0.0000, -5.0000, 11.6000, 11.0000,  6.0000, -0.0000,571.6322,  0.0618, 56.5600
  165:   2.4007,  0.0990, -6.2000, -0.5000,  0.0000, -5.7000,  6.0000,  5.7000,  0.0000,  1.2003,572.8325,  0.0792, 56.6392
  166:   2.4107,  0.0860, -6.5000, -6.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 15.6694,588.5019,  0.5590, 57.1982
  167:   2.1943,  0.1030, -6.8000, -6.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 14.9214,603.4233,  0.7004, 57.8986


In [12]:
DP2bat = Battery(max_capacity=bat_capacity)

DP2 = DPModel2(N, Start, End, house, merged, deepcopy(DP2bat))

J1, pi = DP_stochastic(DP2)

0   

In [13]:
J, traj, actions2 = policy_rollout(DP2,pi=lambda x, k: pi[k][x],x0=0.0)
J

54.1133

In [14]:
series_DP2_battery = logic_rollout(merged.loc[Start:End].copy(), DP2bat, logic_DP, actions2)

charge_list_DP2_bat = DP2bat.charge_list

avg_yearly_cost_dict["DP2"] = (print_price_summary(series_DP2_battery), 
                                      np.sum(charge_list_DP2_bat[charge_list_DP2_bat>0]), 
                                      np.sum(charge_list_DP2_bat[charge_list_DP2_bat<=0]))

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  651.0  DKK
Total emissions for period:  54.0  kg
Average cost per year is: 34190.0 DKK
Average emissions per year is: 2840.0 kg
Number of kwh purchased in the period: 391.2
Average number of kwh purchased per year: 20534.48622754491
Average number of kwh sold per year: 29678.421556886224


In [15]:
logic_series_print(series_DP2_battery)

hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 17.3849, 25.8571,  1.2546,  2.0081
    2:   1.2313,  0.1460, -4.2000, -1.4000,  0.0000, -2.8000,  7.0000,  6.6000,  3.8000,  1.7238, 27.5809,  0.2628,  2.2709
    3:   1.0400,  0.1550, -3.8000, -0.2000,  0.0000, -3.6000,  3.8000,  3.6000,  0.0000,  0.2080, 27.7889,  0.0620,  2.3329
    4:   0.8971,  0.1550, -3.8000, -9.8000,  6.0000,  6.0000,  0.0000,  0.0000,  6.0000,  8.7920, 36.5809,  1.5190,  3.8519
    5:   0.8254,  0.1850, -3.4000, -7.7000,  4.3000,  4.3000,  6.0000,  5.7000, 10.0000,  6.3558, 42.9367,  1.4800,  5.3319
    6:   0.8373,  0.2410, -3.5000,  0.0000,  0.0000, -3.5000, 10.0000,  9.5000,  6.0000, -0.0000, 42.9367,  0.1205,  5.4524
    7: